# Рекомендательная система

**Заказчик.** Компания «Самокат»

**Цель Заказчика.** Получить инструмент, который будет выдавать в рекомендацию 10 аналогичных товаров.

**Цель исследования.** Обучить модель классифицировать товаров со значением метрики качества recall@10 не меньше 0.8.

**Задачи:**

- Загрузить данные.
- Проверить данные на мультиколлиниарность.
- Обучить модель.
- Сделать выводы.

**Входные данные от Заказчика.** Четыре файла в формате .csv с векторным представлением товаров

**Ожидаемый результат.** Построена модель для выдачи на каждый товар запроса десяти аналогичных товаров со значением метрики качества `recall@10` более 0.8

**Описание данных**

1. `base.csv` - векторное представление товаров `base`
2. `train.csv` - векторное представление товаров `query` + таргет (товар из base, являющийся матчем)
3. `test.csv` - векторное представление товаров `query`, для которых надо предсказать кандидатов на матч из base
4. `answer_sample.csv` - формат ответа: `Id - id продукта, Prtedicted - 10 id продуктов из base через пробел`

Работа выполнена на ресурсе `Kaggle`

**Уточнения и ограничения**

При выполнении задания не удалось установить библиотеку `faiss`

Типовые методы (`numpy` и `KMeans`) не позволили выполнить работу на полном датасете, что не позволило провести качественное обучение модели и достигнуть целевого значения заданной метрики.


In [4]:
# Импорт библиотек и выгрузка данных

import numpy as np 
import pandas as pd 
from scipy.spatial import distance
from sklearn.cluster import KMeans

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/samokattechworkshop/base.csv
/kaggle/input/samokattechworkshop/train.csv
/kaggle/input/samokattechworkshop/test.csv
/kaggle/input/samokattechworkshop/baseline.ipynb
/kaggle/input/samokattechworkshop/answer_sample.csv


In [158]:
# чтение файла train
df_train = pd.read_csv('/kaggle/input/samokattechworkshop/train.csv')
df_train.head()

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


**Проверка на мультиколлиниарность**

In [29]:
# Создание матрицы коэффициентов корреляции
corr_tabl = df_train.corr()
corr_tabl = corr_tabl.replace(1,0)

/tmp/ipykernel_34/2094373746.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_tabl = df.corr()


In [30]:
corr_tabl.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.000000,0.005141,0.002172,-0.012514,-0.019630,-0.010645,0.009241,-0.010087,-0.009149,0.000220,...,0.002459,-0.026126,-0.009391,-0.001189,-0.010962,-0.003895,-0.009693,-0.027023,-0.016043,-0.017728
1,0.005141,0.000000,-0.008741,0.005383,-0.000338,-0.008188,0.011282,0.010787,-0.001937,-0.000510,...,-0.005167,-0.003761,-0.021873,-0.004735,0.008024,-0.007563,-0.011825,0.016123,-0.014883,0.020817
2,0.002172,-0.008741,0.000000,-0.014821,-0.017966,0.005171,-0.007881,-0.018625,0.020178,0.016323,...,-0.028888,-0.008702,0.001787,-0.002327,-0.026570,-0.010198,-0.019652,0.002814,0.005712,0.004332
3,-0.012514,0.005383,-0.014821,0.000000,0.009271,-0.007097,-0.004944,0.006825,0.004244,0.033505,...,0.002490,-0.024133,-0.008766,-0.001255,0.012300,0.004461,0.026721,-0.003116,0.005844,0.014606
4,-0.019630,-0.000338,-0.017966,0.009271,0.000000,-0.006796,0.020441,-0.007682,-0.013266,0.016618,...,0.009002,-0.002062,-0.001907,0.002237,-0.003915,-0.011723,-0.005744,-0.012335,0.000445,-0.008475


**Поиск связанных параметров**

In [50]:
corr_tabl[(corr_tabl > 0.5).any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71


In [49]:
corr_tabl[(corr_tabl < -0.5).any(axis=1)]


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71


**Мультиколлиниарность отсутствует**

In [ ]:
# Чтение файла base
df_base = pd.read_csv('/kaggle/input/samokattechworkshop/base.csv')

**Проверка модели на короткой базе**

In [150]:
# Короткая база из 1 000 строк

df_base_short = df_base.loc[:10000]
df_base_short.head()

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [151]:
# Обучающий файл БАЗА (удаление столбца Id)

feature_df_base_short = df_base_short.drop('Id', axis=1)
feature_df_base_short.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


### Основная концепция

1. Методом `KMeans` создать `n` классов.
2. Каждый товар-запрос отнести к какому-либо классу
3. Эвклидовым расстоянием в выбранном классе определить 10 ближайших векторов (товаров)


In [152]:
# Создание модели KMeans 

kmeans = KMeans(n_clusters=500, random_state=0, n_init="auto")
kmeans.fit(feature_df_base_short)

KMeans(n_clusters=500, n_init='auto', random_state=0)

In [159]:
# Добавление столбца с классами в базу

df_base_short['gr_class'] = list(kmeans.labels_)

/tmp/ipykernel_34/330188581.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_short['gr_class'] = list(kmeans.labels_)


In [155]:
df_base_short.head()

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,gr_class
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353,160
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610,433
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206,268
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490,47
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401,109


In [160]:
# Обучающий файл train (удаление столбца Id и Target)

feature_df_train = df_train.drop(['Id', 'Target'], axis=1)

In [161]:
# Обучение модели

df_train['gr_class'] = list(kmeans.predict(feature_df_train))

In [162]:
# Функция создания столбца со списком из класса и индекса

def class_ind_list(cl, ind):
    ind_class = []
    ind_class.append(cl)
    ind_class.append(ind)
    return ind_class

In [163]:
# создания столбца с осписком из класса и индекса

# столбец с индексом
df_train['ind'] = df_train.index

# создание столбца с осписком из класса и индекса

df_train['class_ind'] = df_train.apply(lambda x: ind_class_list(x['gr_class'], x['ind']),
                           axis=1)

In [164]:
df_train.head()

,Id,0,1,2,3,4,5,6,7,8,...,66,67,68,69,70,71,Target,gr_class,ind,class_ind
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base,286,0,"[286, 0]"
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base,366,1,"[366, 1]"
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base,258,2,"[258, 2]"
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base,38,3,"[38, 3]"
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base,60,4,"[60, 4]"


In [165]:
# Функция тонкого подбора рекомендаций методом эвклидова расстояния

def accurate_prtdict(clas_ind):
    # Количество лучших значений
    n = 1
    
    # список значений и список лучших ответов
    best_answ = []
    best_base = []
    
    # список индексов из БАЗЫ по данному классу
    l_i = list(df_base_short.loc[df_base_short['gr_class']==clas_ind[0]].index)
    
    # Вектор запроса
    a = feature_df_train.loc[clas_ind[1]]
    
    # расстояния векторов
    for i in l_i:
        b = feature_df_base_short.loc[i]
        d = distance.euclidean(a, b)
        s = df_base_short.loc[i,'Id']
        best_answ.append((s,d))
     
    # Сортируем список по расстоянию в порядке возрастания
    best_answ.sort(key=lambda x: x[1])
    
    # Возвращаем первые n наиболее похожих строк матрицы
    for i in best_answ[:n]:
        best_base.append(i[0])
    
    return best_base

In [166]:
# Поиск ОДНОГО ответа / товара-аналога

df_train['answ'] = df_train['class_ind'].apply(accurate_prtdict)

In [167]:
df_train.head()

,Id,0,1,2,3,4,5,6,7,8,...,67,68,69,70,71,Target,gr_class,ind,class_ind,answ
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base,286,0,"[286, 0]",[6520-base]
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base,366,1,"[366, 1]",[7120-base]
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base,258,2,"[258, 2]",[3503-base]
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base,38,3,"[38, 3]",[1775-base]
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base,60,4,"[60, 4]",[1836-base]


### Выводы

На «короткой» выборке, с применением методов `numpy` и `KMeans`, сформирована модель для подбора любого количества товаров, аналогичных запрошенному. В приведенной функции (для ускорения расчета) рекомендуется **один** товар.

Отсутствие библиотеки `faiss` не позволило реализовать методику на полном датасете. В результате метрика `recall@10` не определена.

**Таким образом,** работа не завершена и обозначила только концептуальные направления дальнейшего исследования.
